In [1]:
import torch
import torch.nn as nn
from dataset import LargeMovieDataset
from torch.utils.data import DataLoader
from gensim.models import Word2Vec
import torch.nn.functional as F


#MODEL



# class CustomRNN(nn.Module):
#     def __init__(self, input_size, hidden_size, num_layers, bidirectional=False):
#         super(CustomRNN, self).__init__()
#         self.num_layers = num_layers
#         self.bidirectional = bidirectional
#         self.rnn = nn.RNN(input_size, hidden_size, num_layers, bidirectional=bidirectional, batch_first=True)
#         self.layer_norm = nn.LayerNorm(hidden_size * 2 if bidirectional else hidden_size)

#     def forward(self, x, lengths):
#         packed_seq = pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
#         output, hidden = self.rnn(packed_seq)
#         output, _ = pad_packed_sequence(output, batch_first=True)
#         output = self.layer_norm(output)
#         return output, hidden
    


# n_hidden = 128
# rnn = RNN(n_letters, n_hidden, n_categories)

#DATALOADER
word_embedding_size = 16
word2vec_model = Word2Vec.load("../models/word2vec_model.model")

train_set = LargeMovieDataset(path="../data/aclImdb/", set="train", embedding_dic=word2vec_model.wv, word_embedding_size=word_embedding_size)
test_set = LargeMovieDataset(path='../data/aclImdb/', set='test', embedding_dic=word2vec_model.wv, word_embedding_size=word_embedding_size)

train_set, val_set = torch.utils.data.random_split(train_set, [int(0.8 * len(train_set)), len(train_set) - int(0.8 * len(train_set))])

train_dataloader = DataLoader(train_set, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_set, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_set, batch_size=32, shuffle=True)

[nltk_data] Downloading package punkt to
[nltk_data]     /home/matthieulucchesi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


max_len: 999
max_len: 1000


In [10]:
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_layer):
        super(Model, self).__init__()
        self.input_size = input_size        
        self.hidden_size = hidden_size
        self.num_layer = num_layer
        self.c1 = nn.RNN(self.input_size, self.hidden_size, num_layers=num_layer, batch_first=True)
        self.c2 = nn.Linear(self.hidden_size, 2)

    def forward(self, x, mask):
        x, hiden = self.c1(x)
        x = x[:, -1, :]
        x = self.c2(x)
        return F.relu(x)
    

hyp_hidensize = 128
num_layer = 1
rnn_model = Model(word_embedding_size, hyp_hidensize, num_layer)

learnong_rate=1e-2
optimizer_rnn = torch.optim.Adam(rnn_model.parameters(), lr=learnong_rate)
criterion_rnn = nn.CrossEntropyLoss()

In [11]:
from utils import train

train(model=rnn_model, train_dataloader=train_dataloader, val_dataloader=val_dataloader, optimizer=optimizer_rnn, criterion=criterion_rnn, num_epochs=2)

Using device: cpu to train



100%|██████████| 157/157 [00:11<00:00, 13.84batch/s, acc=33.33%, epoch=1, loss=0.698, set=val]
